In [ ]:
import numpy as np
import pandas as pd
import random
import time
import math
import os
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter as sg_filter
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import init
from dateutil import parser
from pathlib import Path
import json 
import shutil

from wattile.entry_point import create_input_dataframe, run_model
from wattile.data_reading import read_dataset_from_file
from wattile.time_processing import add_processed_time_columns
from wattile.buildings_processing import _preprocess_data, correct_predictor_columns, correct_timestamps, resample_or_rolling_stats, pad_full_data
PROJECT_DIRECTORY = Path().resolve().parent.parent

In [ ]:
PROJECT_DIRECTORY

### read configs

In [ ]:
"""
For this example, we will be using the default configs.
Check out the docs for an explaination of each config.
"""
##################################################################################
# choose the configs file to use as an input
##################################################################################
# main configs file
with open(PROJECT_DIRECTORY / "wattile" / "configs" / "configs.json", "r") as f:
    configs = json.load(f)
##################################################################################
# # code testing configs file
# with open(PROJECT_DIRECTORY / "tests" / "fixtures" / "test_configs.json", "r") as f:
#     configs = json.load(f)
##################################################################################

configs["data_dir"] = str(PROJECT_DIRECTORY) + "/data"
configs["exp_dir"] = str(PROJECT_DIRECTORY) + "/exp_dir"

configs

### reading data

In [ ]:
# configs["building"] = "Synthetic Site"
# configs["target_var"] = "Synthetic Site Electricity Main Total Power"

configs["building"] = "Cafe"
configs["target_var"] = "Cafe Whole Building Real Power Total"

configs["num_epochs"] = 100

In [ ]:
data = read_dataset_from_file(configs)
data

### process data

In [ ]:
train_df, val_df = create_input_dataframe(configs)

In [ ]:
train_df.shape

### run (train) model

train model here or via IDE

In [ ]:
run_model(configs, train_df, val_df)

### set configs for applying other data to the trained model

In [ ]:
configs["use_case"] = "validation"

### read (or set aside) sample data

- read new data here if necessary
- example below is using the whole set of data instead of splitting between training and validation

In [ ]:
configs["resample_interval"] = "15min"

In [ ]:
data_sample = data.copy()
data_sample = data_sample.loc["2019-05-01":"2019-07-31"]
data_sample

In [ ]:
# assert we have the correct columns and order them
data_sample = correct_predictor_columns(configs, data_sample)

# sort and trim data specified time period
data_sample = correct_timestamps(configs, data_sample)

# Add time-based features
data_sample = add_processed_time_columns(data_sample, configs)

# Add statistics features
data_sample = resample_or_rolling_stats(data_sample, configs)

# Add lag features
configs["input_dim"] = data_sample.shape[1] - 1

if configs["arch_version"] == 4:
    data_sample = pad_full_data(data_sample, configs)

data_sample

### override val_df with sample data

In [ ]:
val_df = data_sample.reset_index(drop=True)
val_df

### re-run model

In [ ]:
run_model(configs, train_df, val_df)